# Import thư viện

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Đọc và xử lý dữ liệu số

In [2]:
def read_data_number(data_path, time_col="thoi_gian"):
    # Đọc dữ liệu từ file CSV
    data = pd.read_csv(data_path)
    
    # Chuyển đổi cột thời gian sang kiểu datetime và đặt làm index
    data[time_col] = pd.to_datetime(data[time_col])
    data.set_index(time_col, inplace=True)
    
    # Tái lấy mẫu và tính toán thống kê
    y_t = data[['Mực nước hồ (m)', 'Lưu lượng đến hồ (m³/s)']].resample("D").mean()
    
    return y_t

In [3]:
def create_dataframe(y_t, time_col="thoi_gian", value_col_1="y_t_1", value_col_2="y_t_2"):
    df = pd.DataFrame({time_col: y_t.index, 
                       value_col_1: y_t['Mực nước hồ (m)'].values, 
                       value_col_2: y_t['Lưu lượng đến hồ (m³/s)'].values})
    return df

In [4]:
def create_dataset(X, y, n_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - n_steps):
        Xs.append(X[i:i+n_steps])
        ys.append(y[i+n_steps])
    return np.array(Xs), np.array(ys)

In [5]:
# Chuẩn hóa dữ liệu
def scaler(df, time_steps):
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X = df[['y_t_1', 'y_t_2']]
    y = df[['y_t_1']]

    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    # Tạo nhãn cho dữ liệu
    X_data, y_data = create_dataset(X, y, time_steps)

    return scaler_X, scaler_y, X_data, y_data

In [6]:
# Khôi phục dữ liệu sau chuẩn hóa
def inverse_transform_data(scaler, y_pred):
    y_pred_predict = scaler.inverse_transform(y_pred)
    y_pred_predict = pd.DataFrame(y_pred_predict)

    return y_pred_predict

In [7]:
# Chuẩn hóa dữ liệu
def split_data(X_data, y_data, val_length, time_steps):
    # Chia dữ liệu thành tập huấn luyện, kiểm tra và test
    X_test, y_test = X_data[-val_length:], y_data[-val_length:]

    # Reshape dữ liệu đầu vào
    X_test = X_test.reshape(X_test.shape[0], time_steps, X_test.shape[2])

    return X_test, y_test

In [8]:
# Đọc dữ liệu đầu vào
path_data = r"E:\CoChau\2025_Fusion_Data\DataSetAnKhe\data_so_AnKhe\AnKhe.csv"
data = read_data_number(path_data, time_col="thoi_gian")

# Tạo DataFrame chứa thời gian và mực nước
y_t = create_dataframe(data, time_col="thoi_gian", value_col_1="y_t_1", value_col_2="y_t_2")

In [9]:
time_steps = 7
scaler_X, scaler_y, X_data, y_data = scaler(y_t, time_steps)


train_length = int(len(X_data) * 0.7)
val_length = int((len(X_data) - train_length) / 2)
X_test, y_test = split_data(X_data, y_data, val_length, time_steps)

In [10]:
y_target = y_t[-val_length:]
y_target

,thoi_gian,y_t_1,y_t_2
1553,2023-04-03,427.603750,18.408750
1554,2023-04-04,428.247500,25.220000
1555,2023-04-05,427.504583,17.703750
1556,2023-04-06,427.929583,12.675417
1557,2023-04-07,427.941250,34.207083
...,...,...,...
1821,2023-12-27,428.105833,3.659583
1822,2023-12-28,428.084583,6.904167
1823,2023-12-29,428.085000,7.337083
1824,2023-12-30,428.030000,4.776667


# Đánh giá ANOVA trên các mô hình

### Gọi các mô hình

In [11]:
from keras.models import load_model

In [15]:
# Mô hình RNN
results_rnn = [] 
for i in range(3):
    model_rnn = load_model(f'./BestModel/best_model_goc_RNN_{i+1}.hdf5')

    y_pred_rnn = model_rnn.predict(X_test)
    y_pred_rnn = inverse_transform_data(scaler_y, y_pred_rnn)

    # Tính mean và std
    y_pred_rnn_mean = y_pred_rnn.mean()
    y_pred_rnn_std = y_pred_rnn.std()
    print(f"Lần {i+1} - Mean: {y_pred_rnn_mean}, Std: {y_pred_rnn_std}")

    results_rnn.append(y_pred_rnn)

9/9 [==============================] - 0s 709us/step
Lần 1 - Mean: 0    428.370087
dtype: float32, Std: 0    0.308101
dtype: float32
9/9 [==============================] - 0s 783us/step
Lần 2 - Mean: 0    428.372192
dtype: float32, Std: 0    0.278872
dtype: float32
9/9 [==============================] - 0s 2ms/step
Lần 3 - Mean: 0    428.358276
dtype: float32, Std: 0    0.309633
dtype: float32


In [16]:
# Mô hình LSTM
results_lstm = []
for i in range(3):
    model_lstm = load_model(f'./BestModel/best_model_goc_LSTM_{i+1}.hdf5')

    y_pred_lstm = model_lstm.predict(X_test)
    y_pred_lstm = inverse_transform_data(scaler_y, y_pred_lstm)

    # Tính mean và std
    y_pred_lstm_mean = y_pred_lstm.mean()
    y_pred_lstm_std = y_pred_lstm.std()
    print(f"Lần {i+1} - Mean: {y_pred_lstm_mean}, Std: {y_pred_lstm_std}")
    
    results_lstm.append(y_pred_lstm)

9/9 [==============================] - 0s 0s/step
Lần 1 - Mean: 0    428.358246
dtype: float32, Std: 0    0.329898
dtype: float32
9/9 [==============================] - 0s 56us/step
Lần 2 - Mean: 0    428.360931
dtype: float32, Std: 0    0.271178
dtype: float32
9/9 [==============================] - 0s 2ms/step
Lần 3 - Mean: 0    428.356781
dtype: float32, Std: 0    0.326583
dtype: float32


In [17]:
# Mô hình GRU
results_gru = []
for i in range(3):
    model_gru = load_model(f'./BestModel/best_model_goc_GRU_{i+1}.hdf5')

    y_pred_gru = model_gru.predict(X_test)
    y_pred_gru = inverse_transform_data(scaler_y, y_pred_gru)

    # Tính mean và std
    y_pred_gru_mean = y_pred_gru.mean()
    y_pred_gru_std = y_pred_gru.std()
    print(f"Lần {i+1} - Mean: {y_pred_gru_mean}, Std: {y_pred_gru_std}")
    
    results_gru.append(y_pred_gru)

9/9 [==============================] - 0s 2ms/step
Lần 1 - Mean: 0    428.372192
dtype: float32, Std: 0    0.313178
dtype: float32
9/9 [==============================] - 0s 3ms/step
Lần 2 - Mean: 0    428.352783
dtype: float32, Std: 0    0.322372
dtype: float32
9/9 [==============================] - 0s 2ms/step
Lần 3 - Mean: 0    428.348602
dtype: float32, Std: 0    0.332251
dtype: float32


### Đánh giá ANOVA với sự khác biệt về giá trị dự báo giữa các mô hình

In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# 1) Chuẩn bị biến 'thoi_gian'
dates = pd.to_datetime(y_target['thoi_gian'])
dates_cat = dates.dt.strftime('%Y-%m-%d').astype('category')

# 2) Flatten mảng dự báo
rnn_1  = np.asarray(results_rnn[0]).ravel()
rnn_2  = np.asarray(results_rnn[1]).ravel()
rnn_3  = np.asarray(results_rnn[2]).ravel()
lstm_1 = np.asarray(results_lstm[0]).ravel()
lstm_2 = np.asarray(results_lstm[1]).ravel()
lstm_3 = np.asarray(results_lstm[2]).ravel()
gru_1  = np.asarray(results_gru[0]).ravel()
gru_2  = np.asarray(results_gru[1]).ravel()
gru_3  = np.asarray(results_gru[2]).ravel()

# 3) Gộp lại thành DataFrame dạng long
def stack_model_values(values_list, model_name):
    """Chuyển 3 mảng value thành long-format DataFrame"""
    dfs = []
    for i, values in enumerate(values_list, start=1):
        df = pd.DataFrame({
            'thoi_gian': dates_cat,
            'model': model_name,
            'value': values,
            'replicate': f'run_{i}'  # ghi nhận lần chạy để theo dõi
        })
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

df_rnn  = stack_model_values([rnn_1, rnn_2, rnn_3], 'RNN')
df_lstm = stack_model_values([lstm_1, lstm_2, lstm_3], 'LSTM')
df_gru  = stack_model_values([gru_1, gru_2, gru_3], 'GRU')

# 4) Gộp toàn bộ lại
df_anova = pd.concat([df_rnn, df_lstm, df_gru], ignore_index=True)
df_anova['model'] = df_anova['model'].astype('category')
df_anova['thoi_gian'] = df_anova['thoi_gian'].astype('category')

# 5) Debug nhanh trước khi ANOVA
print("Shape of df_anova:", df_anova.shape)
print(df_anova.head())
print(df_anova['value'].describe())

# 6) Chạy ANOVA hai chiều CÓ interaction
fit_inter = ols('value ~ C(model) + C(thoi_gian) + C(model):C(thoi_gian)', data=df_anova).fit()
anova_table = sm.stats.anova_lm(fit_inter, typ=2)
print(anova_table)

Shape of df_anova: (2457, 4)
    thoi_gian model       value replicate
0  2023-04-03   RNN  428.295807     run_1
1  2023-04-04   RNN  427.473511     run_1
2  2023-04-05   RNN  428.526306     run_1
3  2023-04-06   RNN  427.324097     run_1
4  2023-04-07   RNN  428.183807     run_1
count    2457.000000
mean      428.361115
std         0.310509
min       427.249023
25%       428.148285
50%       428.397614
75%       428.583618
max       428.901794
Name: value, dtype: float64
                           sum_sq      df           F        PR(>F)
C(model)                 0.040598     2.0   14.851377  4.054255e-07
C(thoi_gian)           234.129162   272.0  629.764255  0.000000e+00
C(model):C(thoi_gian)    0.388758   544.0    0.522843  1.000000e+00
Residual                 2.238837  1638.0         NaN           NaN
